In [1]:
import gym
print(gym.__version__)

0.14.0


In [2]:
import keras
print(keras.__version__)

Using TensorFlow backend.


2.2.4


In [3]:
import random
import math
import numpy as np
from collections import deque

## Setting up the OpenAI Envronment

In [4]:
# Setup OpenAI gym interface
env = gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        
    

[-0.04686164 -0.0072626   0.0322942   0.0174514 ]
[-0.04700689  0.18738169  0.03264323 -0.26486996]
[-0.04325926  0.38202288  0.02734583 -0.54708081]
[-0.0356188   0.18652763  0.01640421 -0.24590889]
[-0.03188825  0.38141149  0.01148603 -0.53337273]
[-0.02426002  0.1861299   0.00081858 -0.23709285]
[-0.02053742  0.38124015 -0.00392328 -0.52951746]
[-0.01291262  0.18617361 -0.01451363 -0.23807334]
[-0.00918914  0.38149986 -0.0192751  -0.53529872]
[-0.00155915  0.57688748 -0.02998107 -0.83399223]
[ 0.0099786   0.38218771 -0.04666091 -0.55088693]
[ 0.01762236  0.18775111 -0.05767865 -0.27326281]
[ 0.02137738  0.38364659 -0.06314391 -0.58356551]
[ 0.02905031  0.57959366 -0.07481522 -0.89545233]
[ 0.04064218  0.38556152 -0.09272427 -0.62719344]
[ 0.04835341  0.581847   -0.10526813 -0.94757873]
[ 0.05999035  0.77821684 -0.12421971 -1.27139467]
[ 0.07555469  0.97468593 -0.1496476  -1.60025426]
[ 0.09504841  1.17123001 -0.18165269 -1.93560778]
[ 0.11847301  1.36777052 -0.22036484 -2.27868053]


/Users/farhanahmed/Downloads/gym/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[ 0.33020651  1.57496554 -0.60484996 -3.323717  ]
[ 0.36170582  1.38491972 -0.6713243  -3.25640282]
[ 0.38940421  1.19570436 -0.73645236 -3.21704467]
[ 0.4133183   1.00716081 -0.80079325 -3.2049884 ]
[ 0.43346152  1.19516566 -0.86489302 -3.61236933]
[ 0.45736483  1.37985274 -0.93714041 -4.01582667]
[ 0.48496188  1.56055197 -1.01745694 -4.41323857]
[ 0.51617292  1.73659631 -1.10572171 -4.80214141]
[ 0.55090485  1.90737598 -1.20176454 -5.17980445]
[ 0.58905237  1.70551375 -1.30536063 -5.34479013]
[ 0.62316264  1.86640947 -1.41225643 -5.69180534]
[ 0.66049083  2.02149241 -1.52609254 -6.01884419]
[ 0.70092068  1.80734141 -1.64646942 -6.29819521]
[ 0.73706751  1.58847258 -1.77243333 -6.61617385]
[ 0.76883696  1.72906499 -1.9047568  -6.86198204]
[ 0.80341826  1.50099641 -2.04199644 -7.25187596]
[ 0.83343819  1.63671735 -2.18703396 -7.42142006]
[ 0.86617254  1.77451044 -2.33546236 -7.54188114]
[ 0.90166275  1.91699465 -2.48629999 -7.60607366]
[ 0.94000264  2.06672389 -2.63842146 -7.60716106]


In [5]:
print(env.action_space) #See what actions are possible within this action space
print(env.observation_space) #See what observations are possible

Discrete(2)
Box(4,)


In [6]:
# training parameters

n_episodes = 1000 # How many episodes do we want to run
n_win_ticks = 195 # Whats the score we want to end the game at
max_env_steps = None 

gamma = 1.0 # discount factor, a consideration of future reward
epsilon = 1.0 #exploration, agent choosing an action it thinks has best longterm effect with a prob 1-(epsilon)
epsilon_min = 0.01
epsilon_decay = 0.995 # we want the exploration factor to reduce over time to the min
alpha = 0.01 # the learning rate, 0 means nothing new is learned and 1 means only take the most recent into consideration
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

# OpenAI Parameters
memory = deque(maxlen = 100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

In [7]:
# Building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#Model definition # Instantiation
model = Sequential()
model.add(Dense(24,input_dim = 4, activation = 'relu'))
model.add(Dense(48, activation = 'relu'))
model.add(Dense(2, activation = 'relu')) # only 2 actions so come down to 2
model.compile(loss = 'mse', optimizer = Adam(lr = alpha , decay = alpha_decay))



Instructions for updating:
Colocations handled automatically by placer.


In [25]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

#pick what to do; if equal to epsilon pick random action, otherwise let model predict
def choose_action(state, epsilon): 
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epilson(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)* epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1,4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        #what our model predicts
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(np.array(x_batch), np.array(y_batch), batch_size = len(x_batch, verbose = 0))
    
    if epsilon > epsilon_min:
              epsilon = * epsilon_decay
        

              
   
              

SyntaxError: invalid syntax (<ipython-input-25-0966ceb8560d>, line 27)